### This is the code to run the process that does the vendor ranking.
    By : Joseph MTV
    
    Input  : The file containing the Material/Vendor combo, that needs to be ranked.
    Output : The file containing the ranked vendors, exported as .csv file to the configured "Output" folder.

### Assumptions:
    => Feed template has an empty row, preceeding the column header. (Changes to this setup, will require changes 
       in AppConfig.)
    => Input feed file is an .xlsx file.  The active sheet to be considered is to be configured in AppConfig.
    => Footer rows will be dropped.
    => Multiple material/vendor combo appearing within the time-period is rolledup by averaging the numbers.
    => Delivery performance scores are computed for Delivery date and Delivered Qty only.
    => Apply the following weighting rules:
        1. Deviation in dates  * 0.4
        2. Deviation in qty    * 0.4
        3. VendorMaterialCount * 0.2
    => Ranking is done on 
        (Deviation in dates  * 0.4) + (Deviation in qty    * 0.4) + (VendorMaterialCount * 0.2)

### [1.00]: Import the required modules.

In [1]:
# Import the required modules.
import os
import re
import wx             # Install it with pip.
import sys
import json
import shutil         # Install it with pip.
import numpy as np
import pandas as pd
from os import path
from datetime import datetime
from sklearn.utils import resample

import matplotlib.pyplot as plt
%matplotlib inline

# Ignore warnings.
import warnings
warnings.filterwarnings('ignore')

### [1.10]: Instantiate the ML pipeline class.

#### [1.11]: Prompt for file upload.

In [2]:
# Function to prompt the user to upload the file.
def get_path(wildcard):
    app = wx.App(None)
    style = wx.FD_OPEN | wx.FD_FILE_MUST_EXIST
    dialog = wx.FileDialog(None, 'Choose the file for ranking vendors.', wildcard=wildcard, style=style)
    if dialog.ShowModal() == wx.ID_OK:
        path = dialog.GetPath()
    else:
        path = None
    dialog.Destroy()
    return path

#### [1.12] : Instantiate the ML Pipleline class

In [3]:
try:
    # Call the funtion to get the input files.
    #s_infile = get_path('*.csv')
    s_infile = get_path('*.xlsx')
    s_infile = s_infile.replace("\\","\\\\")

    # Get the current working directory.
    s_path = os.getcwd()
    s_path = s_path.replace("\\","\\\\")
    s_basefile = os.path.basename(s_infile)
    s_basefile_csv = (os.path.splitext(s_basefile)[0]) + '.csv'

    # Check, if path exists in system path,else add.
    if s_path in os.environ:
        sys.path.append(s_path)

    # Instantiate and Consume the class.
    from classMLModelingPipeline import *
    # Open and read the App Configuration using json.
    with open(s_path + '\\AppConfig.txt') as json_file:
        # Load the App config details.
        data = json.load(json_file)
        # For each entry in json, extract App config parameters.
        for p in data['AppConfig']:
            applConfig = modMLModelingPipeline  (  p['Id'],
                                                    p['Name'],
                                                    p['Source'],
                                                    p['Output'],
                                                    p['TrainedModel'],
                                                    p['ExecutionLog'],
                                                    p['ExecutionLogFileName'],
                                                    p['FeedActiveSheet'],
                                                    p['FeedSkipRows'],
                                                    p['Archive']
                                                  )
            print(applConfig.Source)
    #===== Create Application Directories ============================================================
    s_classApplication  = 'classMLModelingPipeline'
    s_classMethod       = 'CreateAppDirectories'
    s_statusType        = 'Success'
    s_statusDescription = 'Successfull write operation.'
    applConfig.CreateAppDirectories()
    applConfig.WriteToActivityLog(s_classApplication,
                                  s_classMethod,
                                  s_statusType,
                                  s_statusDescription
                                 )
except AttributeError:
    s_statusDescription = 'No file has been choosen!!!!' 
    applConfig.WriteToActivityLog(s_classApplication,
                                  s_classMethod,
                                  s_statusType,
                                  s_statusDescription
                                 )    
    raise Exception(s_statusDescription)
except:
    s_statusDescription = 'Unexpected error : ' 
    s_statusDescription = s_statusDescription + str(sys.exc_info())
    applConfig.WriteToActivityLog(s_classApplication,
                                  s_classMethod,
                                  s_statusType,
                                  s_statusDescription
                                 )    
    raise Exception(s_statusDescription)

C:\Users\HP\DevEnv\Projects\VendorSelection\KPTL_POC\Source\


### [1.20]: Data loading

In [4]:
try:
    s_classMethod       = 'Data Loading'    
    s_statusType        = 'Success'
    feed_active_sheet   = applConfig.FeedActiveSheet
    feed_skiprows       = int(applConfig.FeedSkipRows)
    feed_skiprows_list  = list(range(int(feed_skiprows)))
    if len(feed_skiprows_list) > 0:
        # Read the input file.
        data_xls = pd.read_excel(s_infile, feed_active_sheet, index_col=None,skiprows=feed_skiprows_list)
    else:
        data_xls = pd.read_excel(s_infile, feed_active_sheet, index_col=None)
    # Drop empty rows.
    data_xls.dropna(axis=0,how='any',inplace=True)
    # Get the path to source directory from class.
    s_in_source_file = applConfig.Source
    s_in_source_file = s_in_source_file + s_basefile_csv

    # Convert it to .csv utf-8 format.
    data_xls.to_csv(s_in_source_file, encoding='utf-8')
    
    # Load the data from the csv file into pandas dataframe.
    dfDataVendor = pd.read_csv(s_in_source_file
                               ,parse_dates = ['Doc. Date','Del Date']
                               ,infer_datetime_format = True
                              )
except:
    s_statusType        = 'Error'    
    s_statusDescription = 'Unexpected error : ' 
    s_statusDescription = s_statusDescription + str(sys.exc_info())
    applConfig.WriteToActivityLog(s_classApplication,
                                  s_classMethod,
                                  s_statusType,
                                  s_statusDescription
                                 )    
    raise Exception(s_statusDescription)    

In [5]:
try:
    s_classMethod       = 'Drop Unnamed Column'
    s_statusType        = 'Success'
    # Drop the unwanted column.
    dfDataVendor.drop(columns=['Unnamed: 0'],inplace=True)
except KeyError:
    pass
except:
    s_statusType        = 'Error'
    s_statusDescription = 'Unexpected error : ' 
    s_statusDescription = s_statusDescription + str(sys.exc_info())
    applConfig.WriteToActivityLog(s_classApplication,
                                  s_classMethod,
                                  s_statusType,
                                  s_statusDescription
                                 )    
    raise Exception(s_statusDescription)

### [1.30]: Data pre-processing.

#### [1.31]: Extract VendorCode.

In [6]:
try:
    s_classMethod       = 'Extract Vendor Code'
    s_statusType        = 'Success'    
    # Extract VendorCode from "Supplier/Supplying Plant" column.
    dfDataVendor['VendorCode']  = dfDataVendor['Supplier/Supplying Plant'].str.extract('(\d+)')
    # Alternate way:
    #dfDataVendor['VendorCode']  = dfDataVendor['Supplier/Supplying Plant'].str.extract('([0-9][0-9][0-9][0-9][0-9][0-9]+)')
except:
    s_statusType        = 'Error'
    s_statusDescription = s_statusDescription + str(sys.exc_info())
    applConfig.WriteToActivityLog(s_classApplication,
                                  s_classMethod,
                                  s_statusType,
                                  s_statusDescription
                                 )    
    raise Exception(s_statusDescription)

#### [1.32]: Inspect datatypes and perform validation.

In [7]:
def GetValidExceptionForDates(lstDateColumns, # list of date columns to validate.
                              dfData,          # data frame of the input data.
                              infile_skiprows  # number of rows to skip in the begining from the input file.
                             ):
    # Initialize.
    validationException = ''
    validException_item = ''
    # Corrected Row_index in source file.
    cRowIndex = infile_skiprows + 2
    # Build the Validation Exception message.
    validExceptionPresent = 0
    #validExcept_header = "'ValidationException':"
    validExcept_body   = "'Incorrect_DateFormat' : ["
    validExcept_bodyfooter = "]"
    # Loop through the list of Date columns.
    for itm in lstDateColumns:
        # Validate columns.
        dfDateCheck = pd.to_datetime(dfData[itm],errors='coerce')
        lstDateExceptions = list(dfDateCheck[dfDateCheck.isnull()].index+cRowIndex)
        if len(lstDateExceptions) > 0:
            validExceptionPresent = 1
            validException_item = validException_item + "'" + itm + "':" + str(lstDateExceptions) + ","

    if validExceptionPresent:
        # Remove extra comma.
        validException_item = validException_item[:-1]
        # Build the Validation Exception Message.
        validationException = validExcept_body + validException_item + validExcept_bodyfooter
    return validationException

In [8]:
def GetValidExceptionForNumeric(lstNumericColumns,  # list of date columns to validate.
                                dfData,          # data frame of the input data.
                                infile_skiprows  # number of rows to skip in the begining from the input file.
                               ):
    # Initialize.
    validationException = ''
    validException_item = ''
    # Corrected Row_index in source file.
    cRowIndex = infile_skiprows + 2
    # Build the Validation Exception message.
    validExceptionPresent = 0
    #validExcept_header = "'ValidationException':"
    validExcept_body   = "'Incorrect_NumericFormat' : ["
    validExcept_bodyfooter = "]"
    # Loop through the list of Date columns.
    for itm in lstNumericColumns:
        # Validate columns.
        dfNumericCheck = pd.to_numeric(dfData[itm],errors='coerce')
        lstNumericExceptions = list(dfNumericCheck[dfNumericCheck.isnull()].index+cRowIndex)
        if len(lstNumericExceptions) > 0:
            validExceptionPresent = 1
            validException_item = validException_item + "'" + itm + "':" + str(lstNumericExceptions) + ","

    if validExceptionPresent:
        # Remove extra comma.
        validException_item = validException_item[:-1]
        # Build the Validation Exception Message.
        validationException = validExcept_body + validException_item + validExcept_bodyfooter
    return validationException

In [9]:
try:
    # Consolidate error description to write to log.
    #======= Validate Numeric columns ========================================================================================
    raise_Except = 0
    validationException = ''
    strComma = ''
    validExcept_header = "'ValidationException':"
    s_statusType        = 'ValidationException'    
    s_classMethod       = 'Parse Datatypes'
    # Convert date column's datatype from string to datetime.
    lstNumericCols = ['Sum of PO Quantity','Sum of      Net Price','Sum of PO Value']
    exceptMessage = GetValidExceptionForNumeric(lstNumericColumns = lstNumericCols, # list of numeric columns to validate.
                                                dfData = dfDataVendor,              # data frame of the input data.
                                                infile_skiprows = feed_skiprows     # number of rows to skip in the begining from the input file.
                                               )
    if len(exceptMessage) > 0:
        raise_Except = 1
        s_statusDescription = exceptMessage

    #======= Validate Date columns =====================================================================================
    # set the class name and status.
    # Convert date column's datatype from string to datetime.
    lstDateCols = ['Doc. Date','Del Date']
    # Get the validation exceptions.
    exceptMessage = GetValidExceptionForDates(lstDateColumns = lstDateCols,    # list of date columns to validate.
                                              dfData = dfDataVendor,           # data frame of the input data.
                                              infile_skiprows = feed_skiprows  # number of rows to skip in the begining from the input file.
                                             )
    # If validation exception found.
    if len(exceptMessage) > 0:
        if raise_Except == 1:
            strComma = ','
        raise_Except = 1
        s_statusDescription = s_statusDescription + strComma + exceptMessage
    
    if raise_Except:
        validationException = validExcept_header + '{' + s_statusDescription + '}'
        # Write to log.
        applConfig.WriteToActivityLog(s_classApplication,
                                      s_classMethod,
                                      s_statusType,
                                      s_statusDescription
                                     ) 
        
        raise Exception('Data validation error/s occured. Please check the application log.')
        
except:
    s_statusType        = 'Error'
    s_statusDescription = validationException + str(sys.exc_info())
    applConfig.WriteToActivityLog(s_classApplication,
                                  s_classMethod,
                                  s_statusType,
                                  s_statusDescription
                                 )    
    raise Exception(s_statusDescription)    

#### [1.33]: Export data cleansed, so far to .csv format for inspection.

In [10]:
# Export the date columns to a csv file and manually inspect for anamolies.
dfDataVendor[['Doc. Date','Del Date','VendorCode']].to_csv('Data_Inspect_prediction.csv')

#### [1.34]: Get the latest data for multiple vendor occurence for the given snapshot.

In [11]:
try:
    s_classMethod       = 'Resolve Multiple Material-Vendor combo'
    s_statusType        = 'Success'    
    # Get the latest data for material/combo appearing more than once.
    dfDataVendorMaxDate = dfDataVendor.groupby(['Material', \
                                                'UOM', \
                                                'VendorCode']).agg({'Doc. Date':'max', \
                                                                    'Del Date' : 'max', \
                                                                    'Sum of PO Quantity':'mean', \
                                                                    'Sum of      Net Price':'mean', \
                                                                    'Sum of PO Value' : 'mean' \
                                                                    }) \
                          .reindex(['Doc. Date','Del Date', \
                                    'Sum of PO Quantity', \
                                    'Sum of      Net Price', \
                                    'Sum of PO Value' \
                                   ], axis=1) \
                          .reset_index()
except:
    s_statusType        = 'Error'
    s_statusDescription = str(sys.exc_info())
    applConfig.WriteToActivityLog(s_classApplication,
                                  s_classMethod,
                                  s_statusType,
                                  s_statusDescription
                                 )    
    raise Exception(s_statusDescription)

In [12]:
# Remove rollup of data.
#dfDataVendorMaxDate = dfDataVendor[['Material','UOM','VendorCode','Doc. Date','Del Date','Sum of PO Quantity',
#                                    'Sum of      Net Price','Sum of PO Value']].copy()

#### [1.35]: Get the delivery performance parameters from the pickle.

In [13]:
try:
    s_classMethod       = 'Get the deviation metrics'
    s_statusType        = 'Success'    

    # Get the delivery performance parameters from the pickle folder.
    dfDeliveryPerformanceParameters = applConfig.ReadDataframeFromPickle()
except:
    s_statusType        = 'Error'
    s_statusDescription = str(sys.exc_info())
    applConfig.WriteToActivityLog(s_classApplication,
                                  s_classMethod,
                                  s_statusType,
                                  s_statusDescription
                                 )    
    raise Exception(s_statusDescription)    

In [14]:
# Get the latest data for material/combo appearing more than once.
dfDeliveryPerformanceParameters = \
dfDeliveryPerformanceParameters.groupby(['Material', \
                                         'VendorCode']).agg({'Deviation_DeliveryDate':'mean', \
                                                             'Deviation_DeliveredQty' : 'mean', \
                                                             'Deviation_DeliveredValue':'mean', \
                                                             'MaterialCountByVendor':'max' \
                                                            }) \
                      .reindex(['Deviation_DeliveryDate', \
                                'Deviation_DeliveredQty', \
                                'Deviation_DeliveredValue', \
                                'MaterialCountByVendor' \
                               ], axis=1) \
                      .reset_index()
dfDeliveryPerformanceParameters['Deviation_DeliveryDate'] = dfDeliveryPerformanceParameters['Deviation_DeliveryDate']. \
                                                            round(2)
dfDeliveryPerformanceParameters['Deviation_DeliveredQty'] = dfDeliveryPerformanceParameters['Deviation_DeliveredQty']. \
                                                            round(2)
dfDeliveryPerformanceParameters['Deviation_DeliveredValue'] = dfDeliveryPerformanceParameters['Deviation_DeliveredValue']. \
                                                            round(2)

In [15]:
#dfDeliveryPerformanceParameters[dfDeliveryPerformanceParameters['Material'] == 'HT10-100X100X10']

### [1.40]: Build the final dataframe for ML modelling.

#### [1.41]: Concatenate all the required columns.

In [16]:
try:
    s_classMethod       = 'Finalize the columns in dataframe.'
    s_statusType        = 'Success'    
    # Concat all the columns required for the analysis.
    dfDataVendorFinal = dfDataVendorMaxDate.merge(dfDeliveryPerformanceParameters,on=['Material','VendorCode'],how='inner')

except:
    s_statusType        = 'Error'
    s_statusDescription = str(sys.exc_info())
    applConfig.WriteToActivityLog(s_classApplication,
                                  s_classMethod,
                                  s_statusType,
                                  s_statusDescription
                                 )    
    raise Exception(s_statusDescription)     

In [17]:
dfDeliveryPerformanceParameters.columns

Index(['Material', 'VendorCode', 'Deviation_DeliveryDate',
       'Deviation_DeliveredQty', 'Deviation_DeliveredValue',
       'MaterialCountByVendor'],
      dtype='object')

### [1.50]: Treat categorical variables.

In [18]:
try:
    s_classMethod       = 'Treat Categorical Varaibles'
    s_statusType        = 'Success' 
    # Build the dataframe with the encoded column for the categorical data.
    dfMaterialOHE = pd.get_dummies(dfDataVendorFinal['Material'])
    dfVendorOHE = pd.get_dummies(dfDataVendorFinal['VendorCode'])
    # Add the encoded columns to the existing dataframe.
    dfDataVendorFinal = pd.concat([dfDataVendorFinal,dfMaterialOHE,dfVendorOHE],axis=1)
    # prepare the rank data by hard coding the weights. 
    dfDataVendorFinal['Rank'] = ( (dfDataVendorFinal['Deviation_DeliveryDate']*0.4) + \
                                  (dfDataVendorFinal['Deviation_DeliveredQty']*0.4) + \
                                  (dfDataVendorFinal['MaterialCountByVendor']*0.2) 
                                )
except:
    s_statusType        = 'Error'
    s_statusDescription = str(sys.exc_info())
    applConfig.WriteToActivityLog(s_classApplication,
                                  s_classMethod,
                                  s_statusType,
                                  s_statusDescription
                                 )    
    raise Exception(s_statusDescription)         

### [1.60]: Machine Learning Modelling

#### [1.61]: Split the dataset.

In [19]:
try:
    s_classMethod       = 'Perform Data Split'
    s_statusType        = 'Success' 

    # Copy the dataframe.
    dfDataVendorML = dfDataVendorFinal.copy()

    # Remove features not useful for the modelling.
    featureDrop = ['Doc. Date'
                   ,'Material'
                   ,'VendorCode'
                   ,'UOM'
                   ,'Del Date'
                   ,'Sum of      Net Price'
                   ,'Sum of PO Quantity'
                   ,'Sum of PO Value'
                   ,'Deviation_DeliveredValue'
                   ,'Rank'
                   ,'MaterialCountByVendor'
                   ]
    dfDataVendorML.drop(featureDrop, axis=1,inplace=True)
    # Extract the independent variable.
    X_Input = dfDataVendorML.copy()
except:
    s_statusType        = 'Error'
    s_statusDescription = str(sys.exc_info())
    applConfig.WriteToActivityLog(s_classApplication,
                                  s_classMethod,
                                  s_statusType,
                                  s_statusDescription
                                 )    
    raise Exception(s_statusDescription)             

#### [1.62]: Standardize the data for new query point.

In [20]:
try:
    s_classMethod       = 'Standardize the data'
    s_statusType        = 'Success' 
    
    # Get the trained standardizer from the pickle.
    objType = 'TN'
    standardizer    = applConfig.GetTrainedObjectFromPickle(objType)
    X_unseen_stdzd  = applConfig.standardize_new_data(dfDataVendorML,
                                                      standardizer) 
except:
    s_statusType        = 'Error'
    s_statusDescription = str(sys.exc_info())
    applConfig.WriteToActivityLog(s_classApplication,
                                  s_classMethod,
                                  s_statusType,
                                  s_statusDescription
                                 )    
    raise Exception(s_statusDescription)             

#### [1.64]: Predict on new data

In [21]:
try:
    s_classMethod       = 'Perform Model classification'
    s_statusType        = 'Success' 

    # Get the trained model and calibrated model from the pickle.
    objType = 'TM'
    lr_optimal    = applConfig.GetTrainedObjectFromPickle(objType)
    objType = 'TCM'
    calibratedCCV = applConfig.GetTrainedObjectFromPickle(objType)

    # Get the predictions from the unseen set.
    Y_pred_unseen,Y_pred_proba_unseen = applConfig.GetPredictionsOnUnseenData( lr_optimal,
                                                                               X_unseen_stdzd)

    # Get the calibrated predictions from the unseen set.
    Y_pred_calib_unseen = applConfig.GetCalibratedPredictionsOnUnseenData( calibratedCCV,
                                                                           X_unseen_stdzd)

except:
    s_statusType        = 'Error'
    s_statusDescription = str(sys.exc_info())
    applConfig.WriteToActivityLog(s_classApplication,
                                  s_classMethod,
                                  s_statusType,
                                  s_statusDescription
                                 )    
    raise Exception(s_statusDescription)  

#### [1.65]: Get class probabilities.

In [22]:
# Update class perdictions back to the main dataframe.
# Class 0 ==> Has deviations.
# Class 1 ==> Has no deviations.
dfDataVendorFinal['class_Deviation_score'] = np.round(Y_pred_calib_unseen[:,0],3)
# Update class probabilities back to the main dataframe.
dfDataVendorFinal['class_NoDeviation_score'] = np.round(Y_pred_calib_unseen[:,1],3)

# Compute Is_Deviation flag.
dfDataVendorFinal['IsDeviation'] = 'Yes'
dfDataVendorFinal.loc[( (dfDataVendorFinal['Deviation_DeliveryDate']   <= 0.0) & \
                        (dfDataVendorFinal['Deviation_DeliveredQty']   == 0.0) \
                      ),  \
                     'IsDeviation'] = 'No'

#### [1.67]: Export the result to .csv format.

In [23]:
# Get the path to store the result.
outputFile = applConfig.Output + 'VendorRanking.csv'
# Export it to csv format.
dfDataVendorFinal[['Material','VendorCode','UOM','Doc. Date','Del Date' \
                   ,'Sum of PO Quantity','Sum of      Net Price','Sum of PO Value','MaterialCountByVendor','Rank' \
                   ,'Deviation_DeliveryDate', 'Deviation_DeliveredQty' \
                   ,'class_NoDeviation_score','class_Deviation_score' \
                   ,'IsDeviation']].to_csv(outputFile)